In [1]:
import arviz as az
import pymc as pm
from pymc.math import ge, switch

# Equivalence of Generic and Brand-name Drugs*

Adapted from [Unit 6: equivalence.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/equivalence.odc).

## Problem statement

The manufacturer wishes to demonstrate that their generic drug for a particular metabolic disorder is equivalent to a brand name drug. One of indication of the disorder is an abnormally low concentration of levocarnitine, an amino acid derivative, in the plasma. The treatment with the brand name drug substantially increases this concentration.

A small clinical trial is conducted with 43 patients, 18 in the Brand Name Drug arm and 25 in the Generic Drug arm. The increases in the log-concentration of levocarnitine are in the data below.

The FDA declares that bioequivalence among the two drugs can be established if the difference in response to the two drugs is within 2 units of log-concentration. Assuming that the log-concentration measurements follow normal distributions with equal population variance, can these two drugs be declared bioequivalent within a tolerance +/-2  units?

---
The way the data is set up in the .odc file is strange. It seems simpler to just have a separate list for each increase type.

In [2]:
# fmt: off
increase_type1 = [7, 8, 4, 6, 10, 10, 5, 7, 9, 8, 6, 7, 8, 4, 6, 10, 8, 9]
increase_type2 = [6, 7, 5, 9, 5, 5, 3, 7, 5, 10, 8, 5, 8, 4, 4, 8, 6, 11, 
                  7, 5, 5, 5, 7, 4, 6]
# fmt: on

We're using ```pm.math.switch``` and ```pm.math.eq``` to recreate the BUGS ```step()``` function for the ```probint``` variable.


In [3]:
with pm.Model() as m:
    # priors
    mu1 = pm.Normal("mu1", mu=10, sigma=316)
    mu2 = pm.Normal("mu2", mu=10, sigma=316)
    mudiff = pm.Deterministic("mudiff", mu1 - mu2)
    prec = pm.Gamma("prec", alpha=0.001, beta=0.001)
    sigma = 1 / pm.math.sqrt(prec)

    probint = pm.Deterministic(
        "probint",
        switch(ge(mudiff + 2, 0), 1, 0) * switch(ge(2 - mudiff, 0), 1, 0),
    )

    y_type1 = pm.Normal("y_type1", mu=mu1, sigma=sigma, observed=increase_type1)
    y_type2 = pm.Normal("y_type2", mu=mu2, sigma=sigma, observed=increase_type2)

    # start sampling
    trace = pm.sample(5000)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu1, mu2, prec]


Output()

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 1 seconds.


In [4]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu1,7.330,0.470,6.379,8.227,0.003,0.002,23824.0,16119.0,1.0
mu2,6.200,0.397,5.441,7.006,0.003,0.002,25173.0,15230.0,1.0
prec,0.264,0.059,0.151,0.376,0.000,0.000,24360.0,15299.0,1.0
mudiff,1.130,0.617,-0.108,2.313,0.004,0.003,23422.0,15936.0,1.0
probint,0.922,0.268,0.000,1.000,0.002,0.001,16357.0,16357.0,1.0


BUGS results:

|          | mean   | sd      | MC_error | val2.5pc | median | val97.5pc | start | sample |
|----------|--------|---------|----------|----------|--------|-----------|-------|--------|
| mu[1]    | 7.332  | 0.473   | 0.001469 | 6.399    | 7.332  | 8.264     | 1001  | 100000 |
| mu[2]    | 6.198  | 0.4006  | 0.001213 | 5.406    | 6.199  | 6.985     | 1001  | 100000 |
| mudiff   | 1.133  | 0.618   | 0.00196  | -0.07884 | 1.134  | 2.354     | 1001  | 100000 |
| prec     | 0.2626 | 0.05792 | 1.90E-04 | 0.1617   | 0.2584 | 0.3877    | 1001  | 100000 |
| probint  | 0.9209 | 0.2699  | 9.07E-04 | 0        | 1      | 1         | 1001  | 100000 |

In [5]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Sun Mar 09 2025

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

pytensor: 2.26.4

pymc : 5.19.1
arviz: 0.20.0

